In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv1D, GlobalAvgPool2D, GlobalAveragePooling1D, \
    Dropout, Dense, Activation, Concatenate, Multiply, MaxPool2D, Add,  \
    LSTM, Bidirectional, Conv2D, AveragePooling2D, AveragePooling1D, BatchNormalization, Flatten, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Reshape, Permute, multiply, Lambda, add, subtract, MaxPooling2D, GRU, ReLU, MaxPooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, roc_curve
from keras.models import Model, load_model
from keras import backend as K
from sklearn.model_selection import KFold
from keras.layers import Embedding
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
mpl.use('TkAgg')
import warnings
warnings.filterwarnings("ignore")

2024-06-04 14:28:05.010458: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 14:28:05.045908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 14:28:05.614836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存，按需分配当allow_growth设置为True时，分配器将不会指定所有的GPU内存，而是根据需求增长
session = tf.compat.v1.Session(config=config)

2024-06-04 14:29:50.111072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5016 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
def get_shuffle(dataset,label):    
    #shuffle data
    index = [i for i in range(len(label))]
    np.random.shuffle(index)
    dataset = dataset[index]
    label = label[index]
    return dataset,label 

In [4]:
#读取训练集bert_embedding的CSV文件(6746，1024)
df_train = pd.read_csv('./data_Kbhb/train_15/traditionalTiQv/DDE_Kbhb_15_train.csv')
train = np.array(df_train) 
train=train[:,1:] #表示从第二列开始所有数据
'''
#对数据进行正态标准化
trans = StandardScaler()
train = trans.fit_transform(train)
'''
#训练集标签
train_label = np.array( [1] * 2866 + [0] * 2866 ).astype( np.float32 )
train_label = to_categorical( train_label, num_classes=2 )
train,train_label=get_shuffle(train,train_label)

# 读取测试集bert_embedding的CSV文件(3518，1024)  
df_test = pd.read_csv('./data_Kbhb/train_15/traditionalTiQv/DDE_Kbhb_15_test.csv')
test = np.array(df_test)  
test=test[:,1:] #表示从第二列开始所有数据
'''
#对数据进行正态标准化
test = trans.fit_transform(test)
'''

#测试集标签
test_label = np.array([1] * 318 + [0] * 318).astype(np.float32)
test_label = to_categorical(test_label, num_classes=2)

test,test_label=get_shuffle(test,test_label)

In [5]:
# Performance evaluation
def show_performance(y_true, y_pred):

    TP, FP, FN, TN = 0, 0, 0, 0

    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] > 0.5:
                TP += 1
            else:
                FN += 1
        if y_true[i] == 0:
            if y_pred[i] > 0.5:
                FP += 1
            else:
                TN += 1


    Sn = TP / (TP + FN + 1e-06)

    Sp = TN / (FP + TN + 1e-06)

    Acc = (TP + TN) / len(y_true)

    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + 1e-06)

    return Sn, Sp, Acc, MCC

def performance_mean(performance):
    print('Sn = %.4f ± %.4f' % (np.mean(performance[:, 0]), np.std(performance[:, 0])))
    print('Sp = %.4f ± %.4f' % (np.mean(performance[:, 1]), np.std(performance[:, 1])))
    print('Acc = %.4f ± %.4f' % (np.mean(performance[:, 2]), np.std(performance[:, 2])))
    print('Mcc = %.4f ± %.4f' % (np.mean(performance[:, 3]), np.std(performance[:, 3])))
    print('Auc = %.4f ± %.4f' % (np.mean(performance[:, 4]), np.std(performance[:, 4])))

In [6]:
from tensorflow.keras.callbacks import Callback
#带有warmup的指数衰减学习率
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1 

In [8]:
# Building the model
def build_model(windows=15, weight_decay=1e-4):
    
    '''
    input_1 = Input(shape=(windows,))

    # Word embedding coding
    embedding = Embedding(21, 32, input_length=15)
    x_1 = embedding(input_1)
    print(x_1.shape)
    '''
    print('*' * 30 + ' DDE+warmup ' + '*' * 30)
    input_1 = Input(shape=(400))
    x_1 = K.expand_dims(input_1, -1)
    print(x_1.shape)

    x_1 = Bidirectional(LSTM(32,return_sequences=True,
                             kernel_regularizer = l2(0.0001), 
                             recurrent_regularizer = l2(0.0001),
                             bias_regularizer = l2(0.0001)))(x_1)
    x_1 = Dropout(0.1)(x_1)
    x_1 = Bidirectional(LSTM(24,return_sequences=True,
                             kernel_regularizer = l2(0.01), 
                             recurrent_regularizer = l2(0.01),
                             bias_regularizer = l2(0.01)))(x_1)
    x_1 = Dropout(0.2)(x_1)
    print(x_1.shape)
    
    '''
    se = SE_block()
    x_1 = se(x_1)
    '''
    
    x_1 = MaxPooling1D(pool_size = 2)(x_1)
    x = Flatten()(x_1)

    x = Dropout(0.2)(x)

    x = Dense(units=2, activation="softmax")(x)


    inputs = [input_1]
    outputs = [x]

    model = Model(inputs=inputs, outputs=outputs, name="Kbhb")

    optimizer = Adam(learning_rate=1e-4, epsilon=1e-7)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./duibifangfa_EAAC/model_test.h5')

del model

model = load_model('./duibifangfa_EAAC/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

****************************** DDE+warmup ******************************
(None, 400, 1)


2024-06-04 14:39:48.117918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5016 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 400, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-04 14:39:56.249914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-04 14:39:56.555365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-04 14:39:56.597508: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f451c1ead40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-04 14:39:56.597532: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-04 14:39:56.601186: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 21:45 - loss: 3.1946 - accuracy: 0.4375

2024-06-04 14:39:57.166764: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 14:39:57.188994: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 14:39:57.192104: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 14:39:57.250492: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

162/162 [==============================] - 34s 161ms/step - loss: 2.9206 - accuracy: 0.6287 - val_loss: 2.6625 - val_accuracy: 0.6655
Epoch 2/200
162/162 [==============================] - 25s 156ms/step - loss: 2.4847 - accuracy: 0.6439 - val_loss: 2.2975 - val_accuracy: 0.6655
Epoch 3/200
162/162 [==============================] - 25s 155ms/step - loss: 2.1580 - accuracy: 0.6534 - val_loss: 2.0103 - val_accuracy: 0.6707
Epoch 4/200
162/162 [==============================] - 25s 156ms/step - loss: 1.8995 - accuracy: 0.6563 - val_loss: 1.7805 - val_accuracy: 0.6725
Epoch 5/200
162/162 [==============================] - 25s 155ms/step - loss: 1.6932 - accuracy: 0.6572 - val_loss: 1.5978 - val_accuracy: 0.6725
Epoch 6/200
162/162 [==============================] - 25s 153ms/step - loss: 1.5262 - accuracy: 0.6607 - val_loss: 1.4506 - val_accuracy: 0.6777
Epoch 7/200
162/162 [==============================] - 25s 156ms/step - loss: 1.3942 - accuracy: 0.6636 - val_loss: 1.3337 - val_accurac

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./duibifangfa_EAAC/model_test.h5')

del model

model = load_model('./duibifangfa_EAAC/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

****************************** AAindex+warmup ******************************
(None, 240, 1)


2024-06-04 12:31:58.121202: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5016 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 240, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-04 12:32:06.573087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-04 12:32:06.980305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-04 12:32:07.019703: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8850319b90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-04 12:32:07.019740: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-04 12:32:07.039893: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 23:23 - loss: 3.2300 - accuracy: 0.4688

2024-06-04 12:32:07.809408: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 12:32:07.825577: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 12:32:07.827002: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-04 12:32:07.883127: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

162/162 [==============================] - 27s 110ms/step - loss: 2.9874 - accuracy: 0.5198 - val_loss: 2.7751 - val_accuracy: 0.5923
Epoch 2/200
162/162 [==============================] - 17s 104ms/step - loss: 2.6053 - accuracy: 0.5874 - val_loss: 2.4341 - val_accuracy: 0.6463
Epoch 3/200
162/162 [==============================] - 17s 104ms/step - loss: 2.3007 - accuracy: 0.6159 - val_loss: 2.1571 - val_accuracy: 0.6481
Epoch 4/200
162/162 [==============================] - 17s 104ms/step - loss: 2.0622 - accuracy: 0.6239 - val_loss: 1.9450 - val_accuracy: 0.6446
Epoch 5/200
162/162 [==============================] - 17s 105ms/step - loss: 1.8694 - accuracy: 0.6373 - val_loss: 1.7729 - val_accuracy: 0.6725
Epoch 6/200
162/162 [==============================] - 17s 104ms/step - loss: 1.7151 - accuracy: 0.6435 - val_loss: 1.6431 - val_accuracy: 0.6411
Epoch 7/200
162/162 [==============================] - 17s 104ms/step - loss: 1.5885 - accuracy: 0.6427 - val_loss: 1.5235 - val_accurac

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./duibifangfa_EAAC/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./duibifangfa_EAAC/model_test.h5')

del model

model = load_model('./duibifangfa_EAAC/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

****************************** EAAC+warmup ******************************
(None, 220, 1)


2024-06-03 10:07:47.878978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8029 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 220, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-03 10:07:55.016670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-03 10:07:55.250547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-03 10:07:55.280608: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556438c646f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-03 10:07:55.280636: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-03 10:07:55.284428: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  2/162 [..............................] - ETA: 10s - loss: 3.1970 - accuracy: 0.4688  

2024-06-03 10:07:55.807459: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-03 10:07:55.831942: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-03 10:07:55.835540: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-03 10:07:55.877597: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

162/162 [==============================] - 20s 78ms/step - loss: 2.9556 - accuracy: 0.5648 - val_loss: 2.7268 - val_accuracy: 0.6080
Epoch 2/200
162/162 [==============================] - 12s 74ms/step - loss: 2.5353 - accuracy: 0.5843 - val_loss: 2.3534 - val_accuracy: 0.5889
Epoch 3/200
162/162 [==============================] - 12s 74ms/step - loss: 2.2006 - accuracy: 0.5956 - val_loss: 2.0540 - val_accuracy: 0.5941
Epoch 4/200
162/162 [==============================] - 12s 74ms/step - loss: 1.9306 - accuracy: 0.5954 - val_loss: 1.8104 - val_accuracy: 0.6063
Epoch 5/200
162/162 [==============================] - 12s 74ms/step - loss: 1.7153 - accuracy: 0.5975 - val_loss: 1.6184 - val_accuracy: 0.5889
Epoch 6/200
162/162 [==============================] - 12s 74ms/step - loss: 1.5489 - accuracy: 0.6014 - val_loss: 1.4739 - val_accuracy: 0.6010
Epoch 7/200
162/162 [==============================] - 12s 74ms/step - loss: 1.4176 - accuracy: 0.5979 - val_loss: 1.3569 - val_accuracy: 0.59

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('5 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('model_test.h5')

del model

model = load_model('model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

2024-05-27 10:26:54.280050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10547 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 128)
(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-05-27 10:27:00.345686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-05-27 10:27:00.531418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-27 10:27:00.545829: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa4d4111610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-27 10:27:00.545877: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-27 10:27:00.554394: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 16:11 - loss: 3.2271 - accuracy: 0.4375

2024-05-27 10:27:01.036844: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:27:01.056585: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:27:01.058466: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:27:01.106513: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

...

162/162 [==============================] - 4s 24ms/step - loss: 0.8796 - accuracy: 0.6840 - val_loss: 0.8831 - val_accuracy: 0.7038
Epoch 19/200
162/162 [==============================] - 4s 24ms/step - loss: 0.8630 - accuracy: 0.6853 - val_loss: 0.8647 - val_accuracy: 0.6969
Epoch 20/200
162/162 [==============================] - 4s 24ms/step - loss: 0.8472 - accuracy: 0.6884 - val_loss: 0.8547 - val_accuracy: 0.6986
Epoch 21/200
162/162 [==============================] - 4s 24ms/step - loss: 0.8335 - accuracy: 0.6888 - val_loss: 0.8397 - val_accuracy: 0.6951
Epoch 22/200
162/162 [==============================] - 4s 24ms/step - loss: 0.8217 - accuracy: 0.6912 - val_loss: 0.8259 - val_accuracy: 0.7003
Epoch 23/200
162/162 [==============================] - 4s 24ms/step - loss: 0.8089 - accuracy: 0.6912 - val_loss: 0.8155 - val_accuracy: 0.6934
Epoch 24/200
162/162 [==============================] - 4s 24ms/step - loss: 0.7974 - accuracy: 0.6954 - val_loss: 0.8037 - val_accuracy: 0.702

2024-05-27 10:29:14.259815: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 4s 25ms/step - loss: 0.6996 - accuracy: 0.7127 - val_loss: 0.7108 - val_accuracy: 0.7213
Epoch 35/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6903 - accuracy: 0.7179 - val_loss: 0.7034 - val_accuracy: 0.7213
Epoch 36/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6831 - accuracy: 0.7111 - val_loss: 0.6984 - val_accuracy: 0.7125
Epoch 37/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6739 - accuracy: 0.7197 - val_loss: 0.6994 - val_accuracy: 0.7143
Epoch 38/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6661 - accuracy: 0.7204 - val_loss: 0.6841 - val_accuracy: 0.7125
Epoch 39/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6589 - accuracy: 0.7237 - val_loss: 0.6820 - val_accuracy: 0.7108
Epoch 40/200
162/162 [==============================] - 4s 24ms/step - loss: 0.6504 - accuracy: 0.7247 - val_loss: 0.6726 - val_accuracy: 0.714

2024-05-27 10:44:49.102765: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 25ms/step - loss: 0.4460 - accuracy: 0.8107 - val_loss: 0.4941 - val_accuracy: 0.7814
Epoch 3/200
180/180 [==============================] - 5s 25ms/step - loss: 0.4469 - accuracy: 0.8067 - val_loss: 0.4990 - val_accuracy: 0.7720
Epoch 4/200
180/180 [==============================] - 4s 25ms/step - loss: 0.4459 - accuracy: 0.8107 - val_loss: 0.4953 - val_accuracy: 0.7814
Epoch 5/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4450 - accuracy: 0.8111 - val_loss: 0.4912 - val_accuracy: 0.7877
Epoch 6/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4435 - accuracy: 0.8105 - val_loss: 0.4969 - val_accuracy: 0.7846
Epoch 7/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4434 - accuracy: 0.8104 - val_loss: 0.4931 - val_accuracy: 0.7830
Epoch 8/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4425 - accuracy: 0.8091 - val_loss: 0.4980 - val_accuracy: 0.7814
Epoc

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('5 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('model_test.h5')

del model

model = load_model('model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

2024-05-27 10:04:09.862771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11971 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 64)
(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-05-27 10:04:15.657296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-05-27 10:04:15.857383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-27 10:04:15.870456: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb58c11a9a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-27 10:04:15.870477: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-27 10:04:15.873870: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 15:32 - loss: 3.2079 - accuracy: 0.5625

2024-05-27 10:04:16.328667: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:04:16.348703: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:04:16.357297: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 10:04:16.400792: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

...

162/162 [==============================] - 4s 23ms/step - loss: 0.8834 - accuracy: 0.6778 - val_loss: 0.8875 - val_accuracy: 0.6882
Epoch 19/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8691 - accuracy: 0.6793 - val_loss: 0.8720 - val_accuracy: 0.6829
Epoch 20/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8560 - accuracy: 0.6782 - val_loss: 0.8628 - val_accuracy: 0.6916
Epoch 21/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8425 - accuracy: 0.6803 - val_loss: 0.8480 - val_accuracy: 0.6951
Epoch 22/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8308 - accuracy: 0.6819 - val_loss: 0.8357 - val_accuracy: 0.7003
Epoch 23/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8192 - accuracy: 0.6865 - val_loss: 0.8247 - val_accuracy: 0.7003
Epoch 24/200
162/162 [==============================] - 4s 23ms/step - loss: 0.8077 - accuracy: 0.6931 - val_loss: 0.8118 - val_accuracy: 0.691

2024-05-27 10:06:23.914772: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 4s 24ms/step - loss: 0.7162 - accuracy: 0.6954 - val_loss: 0.7241 - val_accuracy: 0.7021
Epoch 35/200
162/162 [==============================] - 4s 23ms/step - loss: 0.7077 - accuracy: 0.6950 - val_loss: 0.7162 - val_accuracy: 0.7056
Epoch 36/200
162/162 [==============================] - 4s 23ms/step - loss: 0.7016 - accuracy: 0.6919 - val_loss: 0.7091 - val_accuracy: 0.7003
Epoch 37/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6935 - accuracy: 0.6976 - val_loss: 0.7034 - val_accuracy: 0.6934
Epoch 38/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6878 - accuracy: 0.6935 - val_loss: 0.6955 - val_accuracy: 0.6969
Epoch 39/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6807 - accuracy: 0.7012 - val_loss: 0.6925 - val_accuracy: 0.6951
Epoch 40/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6733 - accuracy: 0.7005 - val_loss: 0.6819 - val_accuracy: 0.703

2024-05-27 10:21:14.116749: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 24ms/step - loss: 0.4658 - accuracy: 0.7962 - val_loss: 0.4893 - val_accuracy: 0.7752
Epoch 3/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4658 - accuracy: 0.7976 - val_loss: 0.4950 - val_accuracy: 0.7767
Epoch 4/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4665 - accuracy: 0.7961 - val_loss: 0.4905 - val_accuracy: 0.7830
Epoch 5/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4668 - accuracy: 0.7954 - val_loss: 0.4867 - val_accuracy: 0.7814
Epoch 6/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4650 - accuracy: 0.7975 - val_loss: 0.4924 - val_accuracy: 0.7830
Epoch 7/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4657 - accuracy: 0.7978 - val_loss: 0.4880 - val_accuracy: 0.7814
Epoch 8/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4650 - accuracy: 0.7955 - val_loss: 0.4929 - val_accuracy: 0.7846
Epoc

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('5 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('model_test.h5')

del model

model = load_model('model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

2024-05-27 09:18:20.699879: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11971 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 32)
(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-05-27 09:18:31.324136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-05-27 09:18:31.745994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-27 09:18:31.766470: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5f4c0539e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-27 09:18:31.766527: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-27 09:18:31.789550: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 29:01 - loss: 3.2315 - accuracy: 0.4688

2024-05-27 09:18:32.749232: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 09:18:32.753599: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 09:18:32.753866: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-27 09:18:32.826564: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

...

162/162 [==============================] - 3s 20ms/step - loss: 0.8901 - accuracy: 0.6669 - val_loss: 0.8938 - val_accuracy: 0.6760
Epoch 19/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8758 - accuracy: 0.6677 - val_loss: 0.8783 - val_accuracy: 0.6794
Epoch 20/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8634 - accuracy: 0.6689 - val_loss: 0.8706 - val_accuracy: 0.6672
Epoch 21/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8521 - accuracy: 0.6669 - val_loss: 0.8566 - val_accuracy: 0.6777
Epoch 22/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8415 - accuracy: 0.6689 - val_loss: 0.8465 - val_accuracy: 0.6777
Epoch 23/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8314 - accuracy: 0.6679 - val_loss: 0.8362 - val_accuracy: 0.6760
Epoch 24/200
162/162 [==============================] - 3s 19ms/step - loss: 0.8206 - accuracy: 0.6724 - val_loss: 0.8249 - val_accuracy: 0.682

2024-05-27 09:20:18.777175: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 3s 20ms/step - loss: 0.7355 - accuracy: 0.6731 - val_loss: 0.7422 - val_accuracy: 0.6864
Epoch 35/200
162/162 [==============================] - 3s 21ms/step - loss: 0.7282 - accuracy: 0.6778 - val_loss: 0.7343 - val_accuracy: 0.6829
Epoch 36/200
162/162 [==============================] - 4s 22ms/step - loss: 0.7206 - accuracy: 0.6731 - val_loss: 0.7279 - val_accuracy: 0.6934
Epoch 37/200
162/162 [==============================] - 4s 22ms/step - loss: 0.7130 - accuracy: 0.6780 - val_loss: 0.7194 - val_accuracy: 0.6934
Epoch 38/200
162/162 [==============================] - 4s 23ms/step - loss: 0.7068 - accuracy: 0.6776 - val_loss: 0.7138 - val_accuracy: 0.7003
Epoch 39/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6992 - accuracy: 0.6811 - val_loss: 0.7092 - val_accuracy: 0.6951
Epoch 40/200
162/162 [==============================] - 4s 23ms/step - loss: 0.6925 - accuracy: 0.6844 - val_loss: 0.6998 - val_accuracy: 0.695

2024-05-27 09:37:21.255943: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 24ms/step - loss: 0.4746 - accuracy: 0.7917 - val_loss: 0.4812 - val_accuracy: 0.7767
Epoch 3/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4737 - accuracy: 0.7908 - val_loss: 0.4854 - val_accuracy: 0.7767
Epoch 4/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4752 - accuracy: 0.7912 - val_loss: 0.4825 - val_accuracy: 0.7799
Epoch 5/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4740 - accuracy: 0.7886 - val_loss: 0.4803 - val_accuracy: 0.7814
Epoch 6/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4733 - accuracy: 0.7906 - val_loss: 0.4842 - val_accuracy: 0.7783
Epoch 7/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4715 - accuracy: 0.7927 - val_loss: 0.4815 - val_accuracy: 0.7814
Epoch 8/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4724 - accuracy: 0.7901 - val_loss: 0.4848 - val_accuracy: 0.7752
Epoc